In [1]:
import pandas as pd 
import numpy as np 
from data_reader import *

/Users/sichenglei/Desktop/kaggle/salt/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)

Bad key "ckend" on line 1 in
/Users/sichenglei/.matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution
/Users/sichenglei/Desktop/kaggle/salt/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
train_sents = get_sents("../../data/train.txt")
val_sents = get_sents("../../data/validate.txt")
test_sents = get_sents("../../data/test.txt")

### CRF (F1: 0.967)

In [3]:
def word2features(sent, i):
    word = sent[i][0]

    ##you may add more features
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit()
    }

    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })

    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

X_train = [sent2features(s) for s in train_sents]
Y_train = [sent2labels(s) for s in train_sents]
X_test = [sent2features(s) for s in test_sents]
Y_test = [sent2labels(s) for s in test_sents]


In [4]:
from sklearn_crfsuite import CRF 

crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

crf.fit(X_train, Y_train)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=False, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [5]:
from sklearn_crfsuite import metrics

labels = list(crf.classes_)
y_pred = crf.predict(X_test)
metrics.flat_f1_score(Y_test, y_pred,
                      average='weighted', labels=labels)

0.9667331846190463

In [12]:
preds = [[int(a) for a in line] for line in y_pred]

In [14]:
len(preds[0])

100

In [8]:
np.asarray(y_pred).shape

(3356, 100)

In [7]:
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    Y_test, y_pred, labels=sorted_labels, digits=3
))

             precision    recall  f1-score   support

          0      0.978     0.991     0.984    323645
          1      0.618     0.402     0.487     11955

avg / total      0.965     0.970     0.967    335600



In [28]:
with open('../../outputs/CRF_preds', 'w') as fout:
    with open('../../data/test.txt', 'r') as test:
        test_list = test.readlines()
        for i in range(len(preds)):
            sent = test_list[i].strip().split()
            data_id = int(sent[2])
            pub_id = int(sent[3])

            j = 0
            while j<len(preds[i]):
                while j<len(preds[i]) and preds[i][j]== 0:
                    j+=1
                if j<len(preds[i]) and preds[i][j] == 1:
                    start = j
                    while j+1<len(preds[i]) and preds[i][j+1]==1:
                        j+=1
                    end = j 
                    fout.write(str(start)+' '+str(end)+' '+str(data_id)+' '+str(pub_id)+'\n')
                    j+=1

### BiLSTM (F1: 0.978)

In [29]:
def sent2labels(sent):
    return [int(label) for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]


In [30]:
X_train = [sent2tokens(s) for s in train_sents]
Y_train = [sent2labels(s) for s in train_sents]
X_val = [sent2tokens(s) for s in val_sents]
Y_val = [sent2labels(s) for s in val_sents]
X_test = [sent2tokens(s) for s in test_sents]
Y_test = [sent2labels(s) for s in test_sents]

In [31]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
vocab_size = len(word_index)+1

Using TensorFlow backend.


In [32]:
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)
X_test = tokenizer.texts_to_sequences(X_test)

In [33]:
maxlen = 100

X_train = pad_sequences(X_train, maxlen=maxlen)
X_val = pad_sequences(X_val, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

Y_train = np.asarray(Y_train)
Y_val = np.asarray(Y_val)
Y_test = np.asarray(Y_test)

In [34]:
#labels need to be 3D
Y_train = np.expand_dims(Y_train, axis=2)
Y_val = np.expand_dims(Y_val, axis=2)
Y_test = np.expand_dims(Y_test, axis=2)

In [35]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

emb_dim = 50
input = Input(shape=(maxlen,))
model = Embedding(vocab_size, emb_dim, input_length=maxlen)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(100, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(1, activation='sigmoid'))(model)

In [36]:
model = Model(input, out)

In [42]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 100, 50)           2324800   
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 50)           0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 100, 200)          120800    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 100, 1)            201       
Total params: 2,445,801
Trainable params: 2,445,801
Non-trainable params: 0
_________________________________________________________________


In [38]:
from keras.callbacks import EarlyStopping

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
earlyStop = [EarlyStopping(monitor='val_loss', patience=1)]


In [39]:
history = model.fit(X_train, Y_train, batch_size=64, epochs=1, validation_data=(X_val, Y_val), callbacks=earlyStop) 


Train on 30208 samples, validate on 4196 samples
Epoch 1/1
  896/30208 [..............................] - ETA: 4:55 - loss: 0.5334 - acc: 0.9347

KeyboardInterrupt: 

In [40]:
from sklearn import metrics 

preds = model.predict(X_test)

In [41]:
test = [[1 if y>=0.5 else 0 for y in x] for x in preds]
# test = np.reshape(np.asarray(test), (-1))
# target = np.reshape(Y_test, (-1))

In [43]:
len(np.asarray(test))

3356

In [69]:
metrics.precision_recall_fscore_support(target, test, average=None,
                                              labels=[0, 1])
#output shape:
#precision 
#recall
#fscore
#support

(array([0.98940991, 0.69476342]),
 array([0.98841941, 0.71359264]),
 array([0.98891441, 0.70405216]),
 array([323645,  11955]))

In [70]:
metrics.precision_recall_fscore_support(target, test, average='micro')

(0.9786293206197855, 0.9786293206197855, 0.9786293206197855, None)

### LSTM-CRF (F1: 0.98)

In [44]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional  
from keras_contrib.layers import CRF
import keras

##using the same processed datasets from above

In [45]:
input = Input(shape=(maxlen,))
model = Embedding(vocab_size, emb_dim, input_length=maxlen)(input)
model = Bidirectional(LSTM(50, return_sequences=True, recurrent_dropout=0.1))(model)    
model = TimeDistributed(Dense(50, activation='relu'))(model)
##use CRF instead of Dense
crf = CRF(2)
out = crf(model)

In [46]:
model = Model(input, out)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 100, 50)           2324800   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 100, 100)          40400     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 100, 50)           5050      
_________________________________________________________________
crf_1 (CRF)                  (None, 100, 2)            110       
Total params: 2,370,360
Trainable params: 2,370,360
Non-trainable params: 0
_________________________________________________________________


In [48]:
Y_train_2 = keras.utils.to_categorical(Y_train)
Y_val_2 = keras.utils.to_categorical(Y_val)
Y_test_2 = keras.utils.to_categorical(Y_test)

In [49]:
model.compile(optimizer='adam', loss=crf.loss_function, metrics=[crf.accuracy]) 


In [50]:
history = model.fit(X_train, Y_train_2, batch_size=64, epochs=1, 
                   validation_data=(X_val, Y_val_2))

Train on 30208 samples, validate on 4196 samples
Epoch 1/1
  640/30208 [..............................] - ETA: 6:19 - loss: 0.1375 - acc: 0.9602

KeyboardInterrupt: 

In [51]:
preds = model.predict(X_test)

In [53]:
test = [[np.argmax(y) for y in x] for x in preds]


In [54]:
test_arr = np.asarray(test)

In [55]:
test_arr.shape

(3356, 100)

In [101]:
preds_labels = [np.argmax(d) for sent in preds for d in sent]

In [107]:
len(preds_labels)

335600

In [56]:
test = np.reshape(test_arr, (-1))

In [58]:
metrics.precision_recall_fscore_support(np.reshape(Y_test,(-1)), 
                                        test, 
                                        average=None,
                                        labels=[0, 1])

/Users/sichenglei/Desktop/kaggle/salt/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0.96437723, 0.        ]),
 array([1., 0.]),
 array([0.98186562, 0.        ]),
 array([323645,  11955]))

In [109]:
metrics.precision_recall_fscore_support(np.reshape(Y_test,(-1)), 
                                        preds_labels, 
                                        average='micro')

(0.9804112038140643, 0.9804112038140643, 0.9804112038140643, None)